In [1]:
import os

if "notebooks" in os.getcwd():
    os.chdir("..")

In [2]:
import time
from tqdm.notebook import tqdm

In [3]:
from azure_cosmos_db import *

azure_cosmos = AzureCosmos()

azure_cosmos.list_containers()

[INFO] Cosmos client created


['eventbrite_events',
 'eventBrite_links',
 'shotgun_links',
 'eventBrite_events',
 'dice_events',
 'shotgun_events',
 'eventbrite_links']

# Events, Dice & Shotgun Scraper

In [4]:
SCRAPER_NAME = "eventbrite_links"
INPUT_FILE_PATH = "data/inputs/Sports Links - Sheet1.csv"
SHEET_NAME = ""

In [5]:
azure_cosmos.DATABASE_ID = "Scraper"
azure_cosmos.CONTAINER_NAME = SCRAPER_NAME

azure_cosmos.container = azure_cosmos.initialize_cosmosdb()

[INFO] Cosmos client created


In [6]:
SHEET_NAME, extension = os.path.splitext(os.path.basename(INPUT_FILE_PATH))

if extension==".csv":
    df = pd.read_csv(INPUT_FILE_PATH)
elif extension==".xlsx":
    df = pd.read_excel(INPUT_FILE_PATH, sheet_name="Sheet4")

print(f"Total: {df.shape}")
df.drop_duplicates(inplace=True)
print(f"Unique: {df.shape}")

print(SHEET_NAME)
# df = df.sample(5)
df.head()

Total: (300, 1)
Unique: (300, 1)
Sports Links - Sheet1


,Links
0,https://www.eventbrite.com/d/united-states/pai...
1,https://www.eventbrite.com/d/united-states/pai...
2,https://www.eventbrite.com/d/united-states/pai...
3,https://www.eventbrite.com/d/united-states/pai...
4,https://www.eventbrite.com/d/united-states/pai...


In [7]:
max_retries = 3
success = 0

urls = df["Links"].to_list()

for url in tqdm(urls, desc=f"Uploading {len(urls)} URLs to {SCRAPER_NAME}"):
    secure_url = url.replace("http://", "https://")
    hash_key = SHEET_NAME + secure_url
    data = {
        "id": hashlib.sha256(hash_key.encode()).hexdigest(),
        "url": secure_url,
        "processed": False,
        "sheet_name": SHEET_NAME
    }

    retries = 0
    while retries < max_retries:
        try:
            azure_cosmos.create_conversation(conversation_data=data)
            success += 1
            break
        except CosmosResourceExistsError as e:
            success += 1
            print(f"[INFO] Record already exists in {azure_cosmos.CONTAINER_NAME}, skipping insertion.")
            break
        except CosmosHttpResponseError as e:
            if e.status_code == 429:
                retry_after = int(e.headers.get("x-ms-retry-after-ms", 1000))/1000
                print(f"[WARNNING] Rate limit exeeded. Retrying in {retry_after}")
                time.sleep(retry_after)
                retries += 1
            else:
                print(e)
                break
                     
print(f"Out of {df.shape[0]} URLs, {success} were uploaded and {df.shape[0]-success} failed to upload")

Uploading 300 URLs to eventbrite_links:   0%|          | 0/300 [00:00<?, ?it/s]

[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversation created successfully
[INFO] Conversat